spark.catalog.clearCache()

In [0]:
import pyspark

In [0]:
spark.conf.set(
    "fs.azure.account.key.datacohortworkspacelabs.dfs.core.windows.net",
    "NUwhjFcHG95EU1Rnu7+Woq3JQP28bXy5kDQhA9yFV68XBz1umr7uqeQgMhrwxHfTwNWxAx/n1K6j+AStGTUMkQ==")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("ApplySchemaAndConstraints").getOrCreate()

bronze_path = "abfss://bronze-kay@datacohortworkspacelabs.dfs.core.windows.net/"
silver_path = "abfss://silver-kay@datacohortworkspacelabs.dfs.core.windows.net/"

bronze_data = spark.read.format("delta").load(bronze_path)

schema = [
    "tripID", "VendorID", "PULocationID", "DOLocationID", 
    "tpep_pickup_datetime", "tpep_dropoff_datetime", 
    "trip_distance", "total_amount","fare_amount", "tip_amount", "payment_type"
]

silver_data = bronze_data.select(*schema)

silver_data = silver_data.filter((col("total_amount").isNotNull()) & (col("total_amount") > 0) & (col("trip_distance") > 0))

silver_data.write.partitionBy("PULocationID").mode("overwrite").format("delta").save(silver_path)

spark.stop()